In [11]:
using DataFrames, CSV, NamedArrays
raw = CSV.read("crops1.csv");
# turn Crops into an array
crops_array = convert(Array,raw);
# the `names' of the DataFrame (header) are the information of each crop;
info_crops = names(raw[2:end]);
# create a list of crops from the crops array
crops = crops_array[1:end,1];
# create a NamedArray that specifies the information of each crop
#crops=convert(Array{Symbol},crops )
using NamedArrays
crops_matrix = crops_array[1:end,2:end]; # rows are crops, columns are information of crops
crops_information_array = NamedArray(crops_matrix, (crops, info_crops), ("crops","info_crops"));


raw1 = CSV.read("LiveStock1.csv");
# turn LiveStock into an array
lstocks_array = convert(Array,raw1);
# the `names' of the DataFrame (header) are the information of each livestock;
info_lstocks = names(raw1[2:end]);
# create a list of stocks from the livestocks array
lstocks = lstocks_array[1:end,1];
# create a NamedArray that specifies the information of each livestock
#lstocks=convert(Array{Symbol},lstocks )
lstocks_matrix = lstocks_array[1:end,2:end] ;# rows are livestocks, columns are information of livestocks
lstocks_information_array = NamedArray(lstocks_matrix, (lstocks, info_lstocks), ("lstocks","info_lstocks"));

#crops to feed our livestocks
feedcrops= [:Wheat, :Corn, :Soybean, :Hay];

#the properties of our farm
tt_field=5;
tt_housing=3;

using JuMP, Clp
m = Model(solver=ClpSolver()) # create model named m

#variables are the area(acre) to grow each crop and the number(hd) for each livestock
@variable(m, crop[crops] >= 0)
@variable(m, lstock[lstocks]>=0)

println(crops_information_array)
println()
println(crops)
println(lstocks)
println(info_lstocks)
#constraints of pasture and field
@constraint(m, sum(crop[i] for i in crops) +sum(lstock[j]*lstocks_information_array[j,3] for j in lstocks)<= tt_field )

#constraints of housing area
@constraint(m, tthousing, sum(lstock[j]*lstocks_information_array[j,1] for j in lstocks) <= tt_housing )

#constraints for feeding the livestocks using our own crops
#@constraint(m, constr[k in feedcrops],sum(lstock[j]*lstocks_information_array[j,k] for j in lstocks) <= crops_information_array[k,2] )

# expression of the total return from livestocks
@expression(m, tot_return_lstock, sum(lstock[i]* lstocks_information_array[i,9 ]for i in lstocks) )
#expression of the total return from crops
@expression(m,   tot_return_crop,sum(crop[i]* (crops_information_array[i,2 ]-crops_information_array[i,1 ])for i in crops) )


#objective: to maximize the return
@objective(m, Max, tot_return_crop+tot_return_lstock)

solve(m)
println(getobjectivevalue(m))
println(getvalue(crop))
println(getvalue(lstock))

13×2 Named Array{Nullable,2}
crops ╲ info_crops │   :Cost  :Return
───────────────────┼─────────────────
"Corn"             │  257.65   565.12
"Soybean"          │  158.14  386.908
"Wheat"            │  159.57  217.147
"Oat"              │  119.38  143.144
"Barley"           │  135.24   323.07
"Flaxseed"         │  130.77   142.41
"Rye"              │  225.65  161.025
"Peas"             │  116.83   229.69
"Beans"            │  239.63   401.96
"Canola"           │  187.14   327.18
"Sunflower"        │  233.22   580.68
"Hops"             │  4000.0  15672.0
"Hay"              │  147.28   322.08

Nullable["Corn", "Soybean", "Wheat", "Oat", "Barley", "Flaxseed", "Rye", "Peas", "Beans", "Canola", "Sunflower", "Hops", "Hay"]
Nullable["Yearling_Steers", "Steers_Calves", "Dairy_Goat", "Pig", "Sheep", "Broiler_organic", "Broiler_regular"]
Symbol[:Enclosed_Housing, :Exercise_Yard, :Pasture, :Corn, :Soybean, :Wheat, :Hay, :Other_Costs, :Returns]


LoadError: [91mMethodError: no method matching *(::JuMP.Variable, ::Nullable{Float64})[0m
Closest candidates are:
  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:424
  *([91m::Nullable{Union{}}[39m, ::Nullable{S}) where S at /nobackup/julia/JuliaPro-0.6.2.2/JuliaPro/pkgs-0.6.2.2/v0.6/NullableArrays/src/operators.jl:108
  *(::JuMP.Variable, [91m::JuMP.Variable[39m) at /nobackup/julia/JuliaPro-0.6.2.2/JuliaPro/pkgs-0.6.2.2/v0.6/JuMP/src/operators.jl:56
  ...[39m